<h1>title<span class="tocSkip"></span></h1>
<div class="toc">undefined</div>

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.png)

# Register model and deploy as webservice

Following this notebook, you will:

 - Learn how to register a model in your Azure Machine Learning Workspace.
 - Deploy your model as a web service in an Azure Container Instance.

In [3]:
import azureml.core
from azureml.core import Workspace

# Check core SDK version number.
print('SDK version:', azureml.core.VERSION)

SDK version: 1.0.76


## Initialize workspace

Create a [Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py) object from your persisted configuration.

In [4]:
import os

# this will be backend script
subscription_id = os.getenv("SUBSCRIPTION_ID", default="c00d16c7-6c1f-4c03-9be1-6934a4c49682")

# if we want user to provide a resource_group then we must ask him. or we create a default resource group for him
resource_group = os.getenv("RESOURCE_GROUP", default="ashkuma_devtesting")

# this is not required if we intend to maintain a workspace for him. i.e. 
# on his first time we create a workspace for him if he doesn't have and then later use the same for subsequent work
workspace_name = os.getenv("WORKSPACE_NAME", default="ashkumamlworkspace")

# region can be fully automated with we selecting one region always.
workspace_region = os.getenv("WORKSPACE_REGION", default="southcentralUS")

from azureml.core import Workspace

#based on his subscription first check if he has any workspaces 
# if he has workspace we don't need to create a new. 
# assume he doesn't have any workspace then.
Workspace.list(subscription_id, auth=None, resource_group=None)

{'ankhokha-ws': [Workspace.create(name='ankhokha-ws', subscription_id='c00d16c7-6c1f-4c03-9be1-6934a4c49682', resource_group='ankhokha-ml-drift')],
 'ashkumadevtestwkrspace': [Workspace.create(name='ashkumadevtestwkrspace', subscription_id='c00d16c7-6c1f-4c03-9be1-6934a4c49682', resource_group='ashkuma_devtesting')],
 'ashkumamlworkspace': [Workspace.create(name='ashkumamlworkspace', subscription_id='c00d16c7-6c1f-4c03-9be1-6934a4c49682', resource_group='ashkuma_devtesting')],
 'gykuma-MLWS': [Workspace.create(name='gykuma-MLWS', subscription_id='c00d16c7-6c1f-4c03-9be1-6934a4c49682', resource_group='gykuma-RG')]}

In [5]:
workspace_name_new ="ashkumadevtestwkrspace"
ws = None
try:
    #ws = Workspace.from_config()
    
    if ws == None:
        try :
            ws = Workspace(subscription_id = subscription_id, resource_group = resource_group , workspace_name = workspace_name_new) # try to check if workspace exist 
        except:
            ws = Workspace.create(name = workspace_name_new,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      sku = 'basic',
                      exist_ok = True)
    print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')
    ws.write_config() # finally write the config
except:
    printf(" unknown exception")



ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

ashkumadevtestwkrspace
ashkuma_devtesting
southcentralus
c00d16c7-6c1f-4c03-9be1-6934a4c49682
ashkumadevtestwkrspace
ashkuma_devtesting
southcentralus
c00d16c7-6c1f-4c03-9be1-6934a4c49682


## Register input and output datasets

For this example, we have provided a small model (`sklearn_regression_model.pkl` in the notebook's directory) that was trained on scikit-learn's [diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset). Here, you will register the data used to create this model in your workspace.

In [ ]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
datastore.upload_files(files=['./features.csv', './labels.csv'],
                       target_path='sklearn_regression/',
                       overwrite=True)

input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/labels.csv')])

## Register model

Register a file or folder as a model by calling [Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none-).

In addition to the content of the model file itself, your registered model will also store model metadata -- model description, tags, and framework information -- that will be useful when managing and deploying models in your workspace. Using tags, for instance, you can categorize your models and apply filters when listing models in your workspace. Also, marking this model with the scikit-learn framework will simplify deploying it as a web service, as we'll see later.

In [ ]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name='my-sklearn-model',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='0.19.1',             # Version of scikit-learn used to create the model.
                       sample_input_dataset=input_dataset,
                       sample_output_dataset=output_dataset,
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

## Deploy model

Deploy your model as a web service using [Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-). Web services take one or more models, load them in an environment, and run them on one of several supported deployment targets. For more information on all your options when deploying models, see the [next steps](#Next-steps) section at the end of this notebook.

For this example, we will deploy your scikit-learn model to an Azure Container Instance (ACI).

### Use a default environment (for supported models)

The Azure Machine Learning service provides a default environment for supported model frameworks, including scikit-learn, based on the metadata you provided when registering your model. This is the easiest way to deploy your model.

**Note**: This step can take several minutes.

In [ ]:
from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = 'my-sklearn-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

After your model is deployed, perform a call to the web service using [service.run()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#run-input-).

In [ ]:
import json

input_payload = json.dumps({
    'data': [
        [ 0.03807591,  0.05068012,  0.06169621, 0.02187235, -0.0442235,
         -0.03482076, -0.04340085, -0.00259226, 0.01990842, -0.01764613]
    ],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

When you are finished testing your service, clean up the deployment with [service.delete()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#delete--).

In [ ]:
service.delete()

### Use a custom environment (for all models)

If you want more control over how your model is run, if it uses another framework, or if it has special runtime requirements, you can instead specify your own environment and scoring method.

Specify the model's runtime environment by creating an [Environment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py) object and providing the [CondaDependencies](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) needed by your model.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn'
])

When using a custom environment, you must also provide Python code for initializing and running your model. An example script is included with this notebook.

In [ ]:
with open('score.py') as f:
    print(f.read())

Deploy your model in the custom environment by providing an [InferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) object to [Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-).

**Note**: This step can take several minutes.

In [ ]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.exceptions import WebserviceException

service_name = 'my-custom-env-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='score.py',
                                   source_directory='.',
                                   environment=environment)

service = Model.deploy(ws, service_name, [model], inference_config)
service.wait_for_deployment(show_output=True)

After your model is deployed, make a call to the web service using [service.run()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#run-input-).

In [ ]:
import json

input_payload = json.dumps({
    'data': [
        [ 0.03807591,  0.05068012,  0.06169621, 0.02187235, -0.0442235,
         -0.03482076, -0.04340085, -0.00259226, 0.01990842, -0.01764613]
    ]
})

output = service.run(input_payload)

print(output)

When you are finished testing your service, clean up the deployment with [service.delete()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#delete--).

In [ ]:
service.delete()

### Model profiling

You can also take advantage of the profiling feature to estimate CPU and memory requirements for models.

```python
profile = Model.profile(ws, "profilename", [model], inference_config, test_sample)
profile.wait_for_profiling(True)
profiling_results = profile.get_results()
print(profiling_results)
```

### Model packaging

If you want to build a Docker image that encapsulates your model and its dependencies, you can use the model packaging option. The output image will be pushed to your workspace's ACR.

You must include an Environment object in your inference configuration to use `Model.package()`.

```python
package = Model.package(ws, [model], inference_config)
package.wait_for_creation(show_output=True)  # Or show_output=False to hide the Docker build logs.
package.pull()
```

Instead of a fully-built image, you can also generate a Dockerfile and download all the assets needed to build an image on top of your Environment.

```python
package = Model.package(ws, [model], inference_config, generate_dockerfile=True)
package.wait_for_creation(show_output=True)
package.save("./local_context_dir")
```

In [4]:
!az aks app up

ERROR: The command failed with an unexpected error. Here is the traceback:

ERROR: 
Traceback (most recent call last):
  File "C:\Users\VSSADM~1\AppData\Local\Temp\pip-install-umfcm_n2\knack\knack\cli.py", line 206, in invoke
  File "C:\Users\VSSADM~1\AppData\Local\Temp\pip-install-umfcm_n2\azure-cli-core\azure\cli\core\commands\__init__.py", line 603, in execute
  File "C:\Users\VSSADM~1\AppData\Local\Temp\pip-install-umfcm_n2\azure-cli-core\azure\cli\core\commands\__init__.py", line 661, in _run_jobs_serially
  File "C:\Users\VSSADM~1\AppData\Local\Temp\pip-install-umfcm_n2\azure-cli-core\azure\cli\core\commands\__init__.py", line 654, in _run_job
  File "C:\Users\VSSADM~1\AppData\Local\Temp\pip-install-umfcm_n2\six\six.py", line 693, in reraise
  File "C:\Users\VSSADM~1\AppData\Local\Temp\pip-install-umfcm_n2\azure-cli-core\azure\cli\core\commands\__init__.py", line 631, in _run_job
  File "C:\Users\VSSADM~1\AppData\Local\Temp\pip-install-umfcm_n2\azure-cli-core\azure\cli\core\comma